In [1]:
from rich import print
import os

from langchain.agents import create_agent
from langchain.agents.middleware import HumanInTheLoopMiddleware 
from langchain.chat_models import init_chat_model
from langchain.tools import tool

from langgraph.checkpoint.memory import InMemorySaver
from langgraph.types import Command

In [2]:
@tool('write_file')
def write_file_tool(file_path: str, content: str) -> str:
    """
    Write content to a file
    
    Args:
        file_path: The path to the file to write to
        content: The content to write to the file
    """

    return f"File written to {file_path} successfully"

@tool('execute_sql')
def execute_sql_tool(query: str) -> str:
    """
    Execute a SQL query
    
    Args:
        query: The SQL query to execute
    """
    
    return f"SQL {query} executed successfully"

@tool('read_data')
def read_data_tool(query: str, limit: int = 10) -> str:
    """
    Read data from a database
    
    Args:
        query: The SQL query to execute
        limit: The maximum number of rows to return
    """
    
    return f"{limit} rows of data read successfully by {query}"

In [3]:
model = init_chat_model(
    "anthropic.claude-3-5-sonnet-20240620-v1:0",
    model_provider="bedrock_converse",
)

agent = create_agent(
    model=model,
    tools=[write_file_tool, execute_sql_tool, read_data_tool],
    middleware=[
        HumanInTheLoopMiddleware( 
            interrupt_on={
                "write_file": True,  # All decisions (approve, edit, reject) allowed
                "execute_sql": {"allowed_decisions": ["approve", "reject"]},  # No editing allowed
                # Safe operation, no approval needed
                "read_data": False,
            },
            # Prefix for interrupt messages - combined with tool name and args to form the full message
            # e.g., "Tool execution pending approval: execute_sql with query='DELETE FROM...'"
            # Individual tools can override this by specifying a "description" in their interrupt config
            description_prefix="Tool execution pending approval",
        ),
    ],
    # Human-in-the-loop requires checkpointing to handle interrupts.
    # In production, use a persistent checkpointer like AsyncPostgresSaver.
    checkpointer=InMemorySaver(),  
)

In [5]:
from langgraph.types import Command

# Human-in-the-loop leverages LangGraph's persistence layer.
# You must provide a thread ID to associate the execution with a conversation thread,
# so the conversation can be paused and resumed (as is needed for human review).
config = {"configurable": {"thread_id": "some_id"}} 
# Run the graph until the interrupt is hit.
result = agent.invoke(
    {
        "messages": [
            {
                "role": "user",
                "content": "請刪除 postgres 中 consumer db 中 orders 資料表， created_at 早於 2024-01-01 的紀錄。",
            }
        ]
    },
    config=config 
)

# The interrupt contains the full HITL request with action_requests and review_configs
print(result['__interrupt__'])

[
    Interrupt(
        value={
            'action_requests': [
                {
                    'name': 'execute_sql',
                    'args': {'query': "DELETE FROM consumer.orders \nWHERE created_at < '2024-01-01';"},
                    'description': 'Tool execution pending approval\n\nTool: execute_sql\nArgs: {\'query\': "DELETE
FROM consumer.orders \\nWHERE created_at < \'2024-01-01\';"}'
                }
            ],
            'review_configs': [{'action_name': 'execute_sql', 'allowed_decisions': ['approve', 'reject']}]
        },
        id='48d4476d39eeed7ec82fcc7afd1060a6'
    )
]

In [6]:
# Resume with approval decision
result = agent.invoke(
    Command( 
        resume={"decisions": [{"type": "approve"}]}  # or "reject"
    ), 
    config=config # Same thread ID to resume the paused conversation
)

print(result)

{
    'messages': [
        HumanMessage(
            content='請刪除 postgres 中 consumer db 中 orders 資料表， created_at 早於 2024-01-01 的紀錄。',
            additional_kwargs={},
            response_metadata={},
            id='34449d43-683b-45c2-a548-034ca7a3464e'
        ),
        AIMessage(
            content=[
                {
                    'type': 'text',
                    'text': '好的,我了解您想要刪除 postgres 中 consumer 資料庫裡 orders 表格中 created_at 早於 
2024-01-01 的記錄。為了執行這個操作，我們需要使用 SQL DELETE 語句。我會使用 `execute_sql` 
函數來執行這個操作。\n\n讓我們執行以下 SQL 查詢：'
                },
                {
                    'type': 'tool_use',
                    'name': 'execute_sql',
                    'input': {'query': "DELETE FROM consumer.orders \nWHERE created_at < '2024-01-01';"},
                    'id': 'tooluse_5vfi6T2FSi6aM0oV-t15fg'
                }
            ],
            additional_kwargs={},
            response_metadata={
                'ResponseMetadata': {
                    'RequestId': '33c6005e-9a80-4182-816d-886ab4d92b5b',
                    'HTTPStatusCode': 200,
                    'HTTPHeaders': {
                        'date': 'Mon, 15 Dec 2025 05:44:13 GMT',
                        'content-type': 'application/json',
                        'content-length': '656',
                        'connection': 'keep-alive',
                        'x-amzn-requestid': '33c6005e-9a80-4182-816d-886ab4d92b5b'
                    },
                    'RetryAttempts': 0
                },
                'stopReason': 'tool_use',
                'metrics': {'latencyMs': [3467]},
                'model_provider': 'bedrock_converse',
                'model_name': 'anthropic.claude-3-5-sonnet-20240620-v1:0'
            },
            id='lc_run--019b2089-a4bc-7ec2-94c4-371b2014a738-0',
            tool_calls=[
                {
                    'name': 'execute_sql',
                    'args': {'query': "DELETE FROM consumer.orders \nWHERE created_at < '2024-01-01';"},
                    'id': 'tooluse_5vfi6T2FSi6aM0oV-t15fg',
                    'type': 'tool_call'
                }
            ],
            usage_metadata={
                'input_tokens': 607,
                'output_tokens': 186,
                'total_tokens': 793,
                'input_token_details': {'cache_creation': 0, 'cache_read': 0}
            }
        ),
        ToolMessage(
            content="SQL DELETE FROM consumer.orders \nWHERE created_at < '2024-01-01'; executed successfully",
            name='execute_sql',
            id='d99f5c25-1faf-427e-86cb-12df4b64894b',
            tool_call_id='tooluse_5vfi6T2FSi6aM0oV-t15fg'
        ),
        AIMessage(
            content='SQL 語句已成功執行。這個操作刪除了 consumer 資料庫中 orders 表格裡所有 created_at 日期早於 
2024 年 1 月 1 
日的記錄。\n\n請注意，這個操作是不可逆的。如果您需要恢復這些數據，您需要從之前的備份中恢復。\n\n如果您想確認刪除操
作的結果，我們可以執行一個 SELECT 查詢來檢查剩餘的記錄。您是否希望我執行這樣的查詢？或者您還有其他問題或需求嗎？',
            additional_kwargs={},
            response_metadata={
                'ResponseMetadata': {
                    'RequestId': 'f7fffd07-9b04-4ed7-adaa-cbad4a832d7f',
                    'HTTPStatusCode': 200,
                    'HTTPHeaders': {
                        'date': 'Mon, 15 Dec 2025 05:44:24 GMT',
                        'content-type': 'application/json',
                        'content-length': '674',
                        'connection': 'keep-alive',
                        'x-amzn-requestid': 'f7fffd07-9b04-4ed7-adaa-cbad4a832d7f'
                    },
                    'RetryAttempts': 0
                },
                'stopReason': 'end_turn',
                'metrics': {'latencyMs': [3771]},
                'model_provider': 'bedrock_converse',
                'model_name': 'anthropic.claude-3-5-sonnet-20240620-v1:0'
            },
            id='lc_run--019b2089-cedd-7aa0-a271-79251e6cec77-0',
            usage_metadata={
                'input_tokens': 825,
                'output_tokens': 184,
                'total_tokens': 